# Topic Modelling and Text Categorisation

In [ ]:
!pip install -U datasets
!pip install -U simpletransformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

In [ ]:
from datasets import load_dataset
import pandas as pd

ds = load_dataset("AmazonScience/mintaka", 'en')

train_df = pd.concat([pd.DataFrame(ds['train']), pd.DataFrame(ds['validation'])])[['question', 'category']]
test_df = pd.DataFrame(ds['test'])[['question', 'category']]
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

0000.parquet:   0%|          | 0.00/1.45M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/252k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/468k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [ ]:
possible_topics_train = train_df['category'].unique()
possible_topics_test = test_df['category'].unique()
possible_topics = list(set(possible_topics_train) | set(possible_topics_test))
print(possible_topics)
print(len(possible_topics))

['politics', 'sports', 'music', 'movies', 'videogames', 'history', 'geography', 'books']
8


In [ ]:
print(train_df.head)
print(train_df.shape)
print(test_df.head)
print(test_df.shape)

<bound method NDFrame.head of                                                question    category
0     What is the seventh tallest mountain in North ...   geography
1     Which actor was the star of Titanic and was bo...      movies
2     Which actor starred in Vanilla Sky and was mar...      movies
3     What year was the first book of the A Song of ...       books
4              Who is the youngest current US governor?    politics
...                                                 ...         ...
1995  Who launched the third Roman invasion of Britain?     history
1996  When was the first Animal Crossing game released?  videogames
1997  Which award did the first puzzle game by Firep...  videogames
1998  Who was the last prime minister during Queen V...    politics
1999  What is the first Discworld book about the Cit...       books

[16000 rows x 2 columns]>
(16000, 2)
<bound method NDFrame.head of                                                question    category
0     What man was

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

model_args = ClassificationArgs()

model_args.overwrite_output_dir=True
model_args.evaluate_during_training=True
model_args.no_save = True

model_args.max_seq_length=256
model_args.use_early_stopping=True
model_args.early_stopping_delta=0.01
model_args.early_stopping_metric='eval_loss'
model_args.early_stopping_metric_minimize=True
model_args.early_stopping_patience=2
model_args.evaluate_during_training_steps=32
model_args.reprocess_input_data=True
model_args.manual_seed=4
model_args.use_multiprocessing=True
model_args.labels_list=possible_topics
model_args.wandb_project="Topic-Catagorization-Sweep"

In [ ]:
import wandb

def train_model():
  wandb.init()
  model = ClassificationModel("distilbert", "distilbert-base-uncased", num_labels=len(possible_topics), args=model_args, use_cuda=True, sweep_config=wandb.config)
  model.train_model(train_df, eval_df=test_df)

In [ ]:
wandb.login()

sweep_configuration = {
	"method": "grid",
	"metric": {"goal": "minimize", "name": "eval_loss"},
	"parameters": {
		"train_epochs": {"values": [8, 10, 12, 16]},
		"train_batch_size": {"values": [16, 32, 64]},
		"learning_rate": {"values": [1e-6, 5e-5, 1e-5, 5e-4]},
	}
}
sweep_id = input("Please enter an existing sweep id if you want to continue an existing sweep (leave blank for a new sweep): ")
sweep_id = None if sweep_id == "" else sweep_id

if sweep_id is None:
	sweep_id = wandb.sweep(sweep=sweep_configuration, project="Topic-Catagorization-Sweep")

wandb.agent(sweep_id, function=train_model, project='Topic-Catagorization-Sweep')

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: simonas-mickus (d3dsec) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Create sweep with ID: 29dobk15
Sweep URL: https://wandb.ai/d3dsec/Topic-Catagorization-Sweep/sweeps/29dobk15


wandb: Agent Starting Run: i9txuycq with config:
wandb: 	learning_rate: 1e-06
wandb: 	train_batch_size: 16
wandb: 	train_epochs: 8


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

...

  0%|          | 0/32 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:882: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 1:   0%|          | 0/500 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:905: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
